In [40]:
import requests
import pandas as pd
from datetime import datetime
import urllib.parse

In [41]:
api_key = ""

def get_match_ids(puuid, continent):
    api_url = "https://"+continent+".api.riotgames.com/lol/match/v5/matches/by-puuid/"+puuid+"/ids?start=0&count=30&"+"&api_key="+api_key
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids


def get_match_data(match_id, continent):
    api_url = "https://"+continent+".api.riotgames.com/lol/match/v5/matches/"+match_id+"?api_key="+api_key
    resp = requests.get(api_url)
    match_data = resp.json()
    return match_data


def find_player_data(match_data, player_puuid):
    participants = match_data["metadata"]["participants"]
    player_index = participants.index(player_puuid)
    player_data = match_data["info"]["participants"][player_index]
    date = datetime.fromtimestamp(match_data["info"]["gameStartTimestamp"] / 1000)
    return [player_data, date]

In [42]:
def gather_all_data(match_ids, continent, player_puuid):

    data = {
        "date": [],
        "champion": [],
        "kills": [],
        "deaths": [],
        "assists": [],
        "barons": [],
        "dragons": [],
        "firstBlood": [],
        "firstTower": [],
        "win": [],
    }

    for match_id in match_ids:
        match_data = get_match_data(match_id, continent)
        player_data = find_player_data(match_data, player_puuid)

        date = player_data[1]
        champion = player_data[0]["championName"]
        kills = player_data[0]["kills"]
        deaths = player_data[0]["deaths"]
        assists = player_data[0]["assists"]
        firstBlood = player_data[0]["firstBloodKill"]
        firstTower = player_data[0]["firstTowerKill"]
        barons = player_data[0]["baronKills"]
        dragons = player_data[0]["dragonKills"]
        win = player_data[0]["win"]

        data["date"].append(date)
        data["champion"].append(champion)
        data["kills"].append(kills)
        data["deaths"].append(deaths)
        data["assists"].append(assists)
        data["firstBlood"].append(firstBlood)
        data["firstTower"].append(firstTower)
        data["barons"].append(barons)
        data["dragons"].append(dragons)
        data["win"].append(win)

    df = pd.DataFrame(data)
    df["win"] = df["win"].astype(int)
    df.to_csv("data.csv", index=False)

    return df

In [43]:
gameName = 'PNG dyNquedo'
tagLine = "pain"
continent = "americas"

player_puuid_url = "https://"+continent+".api.riotgames.com/riot/account/v1/accounts/by-riot-id/"+urllib.parse.quote(gameName)+"/"+tagLine+"?api_key="+api_key
resp_puuid = requests.get(player_puuid_url)
player_info = resp_puuid.json()
player_puuid = player_info["puuid"]

In [44]:
match_ids = get_match_ids(player_puuid, continent)
gather_all_data(match_ids, continent, player_puuid)

,date,champion,kills,deaths,assists,barons,dragons,firstBlood,firstTower,win
0,2024-02-11 16:26:36.425,Yasuo,12,4,2,0,0,True,False,1
1,2024-02-10 17:50:54.440,Yone,1,0,0,0,0,True,False,1
2,2024-02-04 17:48:54.042,Yone,1,2,0,0,0,False,False,0
3,2024-02-04 17:01:11.065,Akali,14,1,6,0,0,False,False,1
4,2024-02-04 15:58:00.267,Yasuo,12,2,3,0,0,False,True,1
5,2024-02-04 14:49:18.781,Yone,13,9,1,0,0,False,True,0
6,2023-11-24 18:07:55.690,Yone,1,4,0,0,0,False,False,0
7,2023-11-24 17:43:14.536,Akali,2,2,1,0,0,False,False,0
8,2023-11-16 03:35:36.539,Jhin,20,5,13,0,0,False,False,1
9,2023-11-16 02:53:41.714,Brand,10,9,18,0,0,False,False,0
